# TP3 - Views, Updates and Design

The objectives for this TP are:

1. Create and use SQL Views
2. Update database content
3. Design the database schema for a Social Network

___

For the first 2 parts we will use the **`wine.db`** database and the Tables created in TP2.

A reminder of the wine database schema:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


In [3]:
import sqlite3

In [4]:
def printSchema(connection):
    ### Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [5]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)

Database schema:
BUYS:
  0: NV(INT) not null *1
  1: NB(INT) not null *2
  2: LIEU(VARCHAR) not null *3
  3: DATES(DATE) not null *4
  4: QTE(INT) not null
CUSTOMER:
  0: NB(INT) *1
  1: NOM(VARCHAR) not null
  2: PRENOM(VARCHAR)
  3: TYPE(VARCHAR) not null
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
PLACE:
  0: LIEU(VARCHAR) not null *1
  1: REGION(VARCHAR) not null
PRODUCER:
  0: NP(INT) *1
  1: NOM(VARCHAR) not null
  2: PRENOM(VARCHAR)
  3: REGION(VARCHAR) not null
PRODUCES:
  0: NV(INT) not null *1
  1: NP(INT) not null *2
  2: QTE(INT)
WINE:
  0: NV(INT) *1
  1: CRU(VARCHAR) not null
  2: DEGRE(INT)
  3: MILL(NUM) not null


Again, we will use **`%%sql`** magic for our queries

In [6]:
%load_ext sql
%sql sqlite:///wine.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


u'Connected: @wine.db'

Recreate the Normalized Tables from **Master1** and **Master2** as you did in the TP2

In [7]:
%%sql 
DROP TABLE IF EXISTS WINE;
CREATE TABLE WINE(
    NV INT PRIMARY KEY, 
    CRU VARCHAR NOT NULL, 
    DEGRE INT, M
    ILL NUM NOT NULL);
insert into WINE 
    SELECT DISTINCT NV, CRU, DEGRE, MILL 
    FROM Master1 
    where nv is not null

 * sqlite:///wine.db
Done.
Done.
102 rows affected.


[]

In [8]:
%%sql 
DROP TABLE IF EXISTS PRODUCER;
CREATE TABLE PRODUCER(
    NP INT PRIMARY KEY, 
    NOM VARCHAR NOT NULL, 
    PRENOM VARCHAR, 
    REGION VARCHAR NOT NULL);
insert into PRODUCER 
    select distinct NP, NOM, PRENOM, REGION 
    from master1 
    where np is not null 

 * sqlite:///wine.db
Done.
Done.
124 rows affected.


[]

In [9]:
%%sql 
DROP TABLE IF EXISTS PRODUCES;
CREATE TABLE PRODUCES(NV INT NOT NULL, NP INT NOT NULL, QTE INT, 
                      PRIMARY KEY(NV,NP),
                      FOREIGN KEY (NV) REFERENCES WINE (NV),
                      FOREIGN KEY (NP) REFERENCES WINE (NP));
insert into PRODUCES 
    SELECT DISTINCT NV, NP, QTE 
    FROM MASTER1 
    WHERE  NV IS NOT NULL AND NP IS NOT NULL

 * sqlite:///wine.db
Done.
Done.
140 rows affected.


[]

In [74]:
%%sql 
DROP TABLE IF EXISTS CUSTOMER;
CREATE TABLE CUSTOMER(
    NB INT PRIMARY KEY, 
    NOM VARCHAR NOT NULL, 
    PRENOM VARCHAR, 
    TYPE VARCHAR NOT NULL);
insert into CUSTOMER 
    select distinct NB, NOM, PRENOM, TYPE 
    from master2 
    where nb is not null 

 * sqlite:///wine.db
Done.
Done.
100 rows affected.


[]

In [11]:
%%sql DROP TABLE IF EXISTS BUYS;
CREATE TABLE BUYS(
    NV INT NOT NULL, 
    NB INT NOT NULL, 
    LIEU VARCHAR NOT NULL, 
    DATES DATE NOT NULL, 
    QTE INT NOT NULL, 
    PRIMARY KEY(NV,NB,LIEU,DATES),
    FOREIGN KEY (NV) REFERENCES WINE (NV),
    FOREIGN KEY (NB) REFERENCES CUSTOMER (NB));
insert into BUYS 
    select distinct nv, nb, lieu, dates, qte 
    from master2 
    where nv is not null and nb is not null 

 * sqlite:///wine.db
Done.
Done.
73 rows affected.


[]

In [12]:
%%sql 
DROP TABLE IF EXISTS PLACE;
CREATE TABLE PLACE(
    LIEU VARCHAR NOT NULL, 
    REGION VARCHAR NOT NULL,  
    PRIMARY KEY(LIEU),
    FOREIGN KEY (LIEU) REFERENCES BUYS (LIEU));
INSERT INTO PLACE 
    select distinct lieu, region 
    from master2 
    where lieu is not null;

 * sqlite:///wine.db
Done.
Done.
18 rows affected.


[]

___
# PART I: CREATE AND USE VIEWS

A view is a virtual table based on the result-set of an SQL statement. Views are stored in the database with an associated name.

Views are created following the syntax:

```mysql
CREATE VIEW view_name AS
SELECT column1, column2.....
FROM table_name
WHERE [condition];
```

An useful command:

```mysql
DROP VIEW IF EXISTS view_name;
```


__Warning:__ Use `DROP` with caution (only drop something if you are sure)

__Note:__ You will find some cells marked as "Test" that will help you check your work. Do NOT modify them. 

#### Exercise 1.1

Create a view "**bons_buveurs**" with the clients (buveurs) of type 'gros' or 'moyen'.

In [14]:
%%sql
CREATE VIEW bons_buveurs AS
SELECT *
FROM CUSTOMER
WHERE TYPE IN ('moyen','gros')

 * sqlite:///wine.db
Done.


[]

In [17]:
# Exécution
%sql SELECT count(*) FROM bons_buveurs ORDER BY nb;

 * sqlite:///wine.db
Done.


count(*)
56


In [7]:
# Test
%sql SELECT * FROM bons_buveurs ORDER BY nb;

Done.


NB,NOM,PRENOM,TYPE
2,Artaud,Antonin,moyen
3,Aron,Raymond,gros
4,Apollinaire,Guillaume,moyen
6,Arrabal,Fernando,gros
7,Anouilh,Jean,moyen
8,Aragon,Louis,gros
10,Andersen,Yann,gros
12,Bataille,Georges,moyen
13,Barthes,Roland,moyen
14,Bory,Jean Louis,gros


#### Exercise 1.2

Create the view "**buveurs_asec**" with clients (buveurs) who have not bought any wine.

In [21]:
%%sql
CREATE VIEW buveurs_asec AS
SELECT *
FROM CUSTOMER
WHERE NB NOT IN (
    SELECT NB
    FROM BUYS
    )

 * sqlite:///wine.db
Done.


[]

In [23]:
%%sql
SELECT count(*)
FROM buveurs_asec

 * sqlite:///wine.db
Done.


count(*)
57


In [9]:
# Test
%sql SELECT * FROM buveurs_asec ORDER BY nb;

Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


#### Exercise 1.3

Create the view "**buveurs_achats**" complementary to the previous one.

In [26]:
%%sql
CREATE VIEW buveurs_achats AS
SELECT *
FROM CUSTOMER
WHERE NB IN (
    SELECT NB
    FROM BUYS
    )

 * sqlite:///wine.db
(sqlite3.OperationalError) table buveurs_achats already exists [SQL: u'CREATE VIEW buveurs_achats AS\nSELECT *\nFROM CUSTOMER\nWHERE NB IN (\n    SELECT NB\n    FROM BUYS\n    )'] (Background on this error at: http://sqlalche.me/e/e3q8)


In [29]:
%%sql
select count(*) from buveurs_achats

 * sqlite:///wine.db
Done.


count(*)
43


In [11]:
# Test
%sql SELECT * FROM buveurs_achats ORDER BY nb;

Done.


NB,NOM,PRENOM,TYPE
1,Aristote,None,petit
2,Artaud,Antonin,moyen
3,Aron,Raymond,gros
4,Apollinaire,Guillaume,moyen
5,Audiberti,Jacques,petit
6,Arrabal,Fernando,gros
7,Anouilh,Jean,moyen
8,Aragon,Louis,gros
9,Ajar,Emile,petit
10,Andersen,Yann,gros


#### Exercise 1.4

Create the view "**q83pl**" (LIEU, CRU, QTE_BUE) that provides by LIEU and CRU the total quantities bought in 1983 by all the clients (buveurs).

In [42]:
%%sql
CREATE VIEW q83pl AS
SELECT B.LIEU, W.CRU, SUM(B.QTE) AS QTE_BUE
FROM buveurs_achats BV 
    INNER JOIN BUYS B ON (B.NB = BV.NB)
    INNER JOIN WINE W ON (B.NV = W.NV)
WHERE strftime('%Y', DATES) = '1983'
GROUP BY B.LIEU, W.CRU
ORDER BY B.LIEU, W.CRU

 * sqlite:///wine.db
Done.


[]

In [44]:
%sql SELECT * FROM q83pl;

 * sqlite:///wine.db
Done.


LIEU,CRU,QTE_BUE
CAEN,Seyssel,3
LILLE,Pommard,5
LYON,Beaujolais Villages,10
LYON,Julienas,2
PARIS,Beaujolais Primeur,4
PARIS,Coteaux du Tricastin,1
PARIS,Pouilly Vinzelles,3
RENNES,Mercurey,1
ROCQUENCOURT,Beaujolais Villages,260
ROCQUENCOURT,Saint Amour,80


In [13]:
# Test
%sql SELECT * FROM q83pl;

Done.


LIEU,CRU,QTE_BUE
CAEN,Seyssel,3
LILLE,Pommard,5
LYON,Beaujolais Villages,10
LYON,Julienas,2
PARIS,Beaujolais Primeur,4
PARIS,Coteaux du Tricastin,1
PARIS,Pouilly Vinzelles,3
RENNES,Mercurey,1
ROCQUENCOURT,Beaujolais Villages,260
ROCQUENCOURT,Saint Amour,80


#### Exercise 1.5

Can we define the same view with ascending order over the attribute "QTE"? Provide an explanation for your answer.

___
# PART II: UPDATE DATABASE CONTENT

The syntax for the `UPDATE` operation is:

```sql
UPDATE table_name
SET column1 = value1, column2 = value2...., columnN = valueN
WHERE [condition];
```

The syntax for the `INSERT` operation is:

```sql
INSERT INTO table_name [(column1, column2, column3,...columnN)]  
VALUES (value1, value2, value3,...valueN);
```

Database updates are commited automatically in Jupyter/Python. _Transactions_ are an option to control and reverse changes. Additionally we can just reload a backup of the database (NOT an option in deployed systems)

__Note:__ Different to other Database Management Systems, SQLite views are read-only and so you can not execute a `DELETE`, `INSERT` or `UPDATE` statement on a view.

#### Exercise 2.1

Create a table "**RBB**" with the same schema as "**bons_buveurs**" which contains the tuples selected from "**bons_buveurs**"

In [75]:
%%sql 
DROP TABLE IF EXISTS RBB;
CREATE TABLE RBB(
    NB INT PRIMARY KEY, 
    NOM VARCHAR NOT NULL, 
    PRENOM VARCHAR, 
    TYPE VARCHAR NOT NULL);
insert into RBB 
    select * 
    from bons_buveurs 
   

 * sqlite:///wine.db
Done.
Done.
56 rows affected.


[]

In [76]:
# Test
%sql SELECT * FROM RBB;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
27,Dumas,Alexandre,gros
32,Eluard,Paul,moyen
35,Fromentin,Eugene,gros
39,Montesquieu,None,gros
42,Goethe,Johann Wolfgang von,moyen
43,Musset,Alfred de,gros


#### Exercise 2.2

Update the table you used to create "**bons_buveurs**": Change the "type" to 'gros' if the total of quantities bought is over 100.

Find the instances to update (schema may differ from the one in your table)

In [77]:
%%sql
SELECT C.NB, C.NOM, C.PRENOM, C.TYPE, SUM(B.QTE) as total
FROM CUSTOMER C INNER JOIN BUYS B ON C.NB=B.NB
WHERE TYPE IN ('moyen','petit')
GROUP BY C.NB, C.NOM, C.PRENOM, C.TYPE
HAVING total > 100

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE,total
2,Artaud,Antonin,moyen,583
5,Audiberti,Jacques,petit,113
9,Ajar,Emile,petit,140
44,Gide,Andre,petit,171


Done.


NB,NOM,PRENOM,TYPE,total
2,Artaud,Antonin,moyen,583
5,Audiberti,Jacques,petit,113
9,Ajar,Emile,petit,140
44,Gide,Andre,petit,171


Update instances

In [78]:
%%sql
UPDATE CUSTOMER SET TYPE = 'gros'
WHERE NB IN (
SELECT C.NB
FROM CUSTOMER C INNER JOIN BUYS B ON C.NB=B.NB
WHERE TYPE IN ('moyen','petit')
GROUP BY C.NB, C.NOM, C.PRENOM, C.TYPE
HAVING SUM(QTE) > 100)

 * sqlite:///wine.db
4 rows affected.


[]

4 rows affected.


[]

#### Exercise 2.3

Compare the content of _table_ "**RBB**" and the _view_ "**bons_buveurs**" after the update. What differences do you see? Explain

In [79]:
%%sql
SELECT *, '' AS TYPE_ANCIEN, 'ajout' as maj FROM bons_buveurs
WHERE NB NOT IN (
   SELECT NB FROM RBB )
UNION
SELECT bons_buveurs.*,RBB.TYPE AS TYPE_ANCIEN, 'modification' as maj
FROM RBB INNER JOIN bons_buveurs ON RBB.NB = bons_buveurs.NB
WHERE bons_buveurs.TYPE <> RBB.TYPE

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE,TYPE_ANCIEN,maj
2,Artaud,Antonin,gros,moyen,modification
5,Audiberti,Jacques,gros,,ajout
9,Ajar,Emile,gros,,ajout
44,Gide,Andre,gros,,ajout


#### Exercise 2.4

Create a table "**RBA**" with the same schema as "**buveurs_asec**" which contains the tuples selected from "**buveurs_asec**"

In [80]:
%%sql 
DROP TABLE IF EXISTS RBA;
CREATE TABLE RBA(
    NB INT PRIMARY KEY, 
    NOM VARCHAR NOT NULL, 
    PRENOM VARCHAR, 
    TYPE VARCHAR NOT NULL);
insert into RBA 
    select * 
    from buveurs_asec 
   

 * sqlite:///wine.db
Done.
Done.
57 rows affected.


[]

In [19]:
# Test
%sql SELECT * FROM RBA

Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


#### Exercise 2.5

Insert a tuple (101, 'your last name', 'your first name', 'your type of purchases(petit, moyen, or gros)') to "**RBA**".

In [82]:
%%sql
INSERT INTO RBA (NB,NOM,PRENOM,TYPE)
VALUES (101,'Roussel','Matthieu','petit')

 * sqlite:///wine.db
1 rows affected.


[]

1 rows affected.


[]

In [84]:
# Verification
%sql SELECT * FROM RBA ORDER BY NB DESC LIMIT 1

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
101,Roussel,Matthieu,petit


In [21]:
# Test
%sql SELECT * FROM RBA

Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


#### Exercise 2.6

Compare the content of _table_ "**RBA**" and the _view_ "**buveurs_asec**". What differences do you see? Explain

In [85]:
%%sql
SELECT *, 'ajout' as maj FROM RBA
WHERE NB NOT IN (
   SELECT NB FROM buveurs_asec )


 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE,maj
101,Roussel,Matthieu,petit,ajout


___
# PART III: Design the database schema for posts in a Social Network

In this section your task is to design the database schema for a social network app of a new startup:

The new social network will contain users, where each user will have a name, a nickname, an email, date of birth, and an address (Street, City, State, Country, Postal Code). Users can be friends of other users, and can publish posts. Each post can contain a text, date and attachment. Posts can be either original posts or replies so the app needs to handle both scenarios. When users log in, the app needs to display the posts of their friends.

**Note:** You can create diagrams of your proposal and insert them as images into this notebook.

#### Exercise 3.1

Write and explain the design of the relations of your database

Answer:

1. Entities : 

USER (__**nickname**__, name, email, date_of_birth, adr_Street, adr_City, adr_State, adr_Country, adr_Postal_Code) )

Note : I've considered that the nickname was the primary key (ie each nickname would be unique)

POST (__**id**__, nickname, text, date, attachment)

Note : id is a simple UID for a post
nickname is a foreign key inherited from users since a post can only be written by a unique user. A user can write 0 to N post.


2. Relationships

IS_FRIEND(__**nickname**__,__**nickname_friend**__)

Note : primary key si composed of foreign keys nickname from table USER, with roles 'nickname' and 'nickname_friend'
Cardinalities : 
USER - IS_FRIEND = 0..*  on both sides

IS_REPLY(__**id_source**__,__**id_reply**__,)
Note : primary key si composed of foreign keys id from table POST, with roles 'id_source' and 'id_reply'
POST - IS_POST = 0..* for role source and 1,1 for role id_reply 

Note : Since a reply has only one source, we could when creating the post table, indicate a source_id on the post table. If completed, the post is a reply. 


#### Exercise 3.2

Write a view to retrieve the posts to display when a user logs in. Consider that some users may have a lot of friends and you need to limit the number of post to display. How would you select relevant posts to display first? What kind of information would you use/add in the database for this purpose? Explain your answer.

__Note:__ Limiting the number of posts just by count is too simplistic, the user could be missing something interesting to him/her.